In [1]:
%aiida
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import load_node, Code, Computer
from aiida.common.exceptions import NotExistent, MultipleObjectsError

# Install and/or update plugins

In [2]:
! pip install -e ./plugins/
! reentry scan -r aiida

Obtaining file:///project/apps/stm/plugins
  Found existing installation: aiida-stm-plugins 0.0.0
    Uninstalling aiida-stm-plugins-0.0.0:
      Successfully uninstalled aiida-stm-plugins-0.0.0
  Running setup.py develop for aiida-stm-plugins


# Setup codes

In [3]:
## -------------------------------------------------------
## List computers
qb = QueryBuilder()
qb.append(Computer)

for e in qb.all():
    print(e)
## -------------------------------------------------------

[<Computer: daint-s746 (daint.cscs.ch), pk: 1>]
[<Computer: daint-s746-normal (daint.cscs.ch), pk: 2>]


In [4]:
## -------------------------------------------------------
## List  codes
qb = QueryBuilder()
qb.append(Code)

for e in qb.all():
    print(e)
## -------------------------------------------------------

[<Code: Remote code 'cp2k_6.0_18265' on daint-s746, pk: 214, uuid: f29f8268-2d86-4d8c-9fe8-cb491575db42>]
[<Code: Remote code 'pw-6.1' on daint-s746, pk: 215, uuid: 72f53cb2-1725-444a-82ad-4d6b8b6b8d63>]
[<Code: Remote code 'pp-6.1' on daint-s746, pk: 216, uuid: 7828ae51-1a67-4675-97b6-09c35a92e00c>]
[<Code: Remote code 'ph-6.1' on daint-s746, pk: 217, uuid: aef87ad8-91c1-4c4f-a901-05851d3b4a69>]
[<Code: Remote code 'projwfc-6.1' on daint-s746, pk: 218, uuid: 049ab577-eea0-4f6a-bda4-9104c833839e>]
[<Code: Remote code 'matdyn-6.1' on daint-s746, pk: 219, uuid: 75226995-05f2-4f0b-accc-82fde07302fd>]
[<Code: Remote code 'q2r-6.1' on daint-s746, pk: 220, uuid: a59fde03-84a6-4b1a-9333-9ccc63a90c1c>]
[<Code: Remote code 'python3' on daint-s746-normal, pk: 2444, uuid: 8c7308b7-1cf8-4d53-b0d2-02ef70f345da>]
[<Code: Remote code 'cp2k_6.0_18265' on daint-s746-normal, pk: 1078, uuid: 2b831656-7e4e-4ede-b4a1-e7868ed86c34>]
[<Code: Remote code 'pw-6.1' on daint-s746-normal, pk: 1079, uuid: 409de94c

In [5]:
# 1. Select computer

computer_name = "daint-s746-normal"
computer = Computer.get(computer_name)

# 2. Set remote path where to install the codes

code_remote_path = "/users/keimre/aiida-soft"


## Retrieve scripts to remote machine

In [6]:
commit_hash = ! git ls-remote "https://github.com/eimrek/atomistic_tools.git"
version_id = commit_hash[0][:6]

code_dir_name = "at_"+version_id

# ssh key needs to be set

! ssh {computer.hostname} "wget https://github.com/eimrek/atomistic_tools/archive/master.zip -O {code_remote_path}/{code_dir_name}.zip"
! ssh {computer.hostname} "cd {code_remote_path} && unzip -o {code_dir_name}.zip && rm -rf {code_dir_name} && mv atomistic_tools-master {code_dir_name}"
! ssh {computer.hostname} "cd {code_remote_path}/{code_dir_name} && chmod u+x evaluate_morbs_in_box_mpi.py stm_sts_extrap_from_npz.py"

--2018-06-11 23:46:12--  https://github.com/eimrek/atomistic_tools/archive/master.zip
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/eimrek/atomistic_tools/zip/master [following]
--2018-06-11 23:46:12--  https://codeload.github.com/eimrek/atomistic_tools/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.121, 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/users/keimre/aiida-soft/at_0eadfd.zip’

     0K .......... .......... .......... .......... ..........  242K
    50K .......... .......... .......... .......... ..........  481K
   100K .......... .......... .......... .......... ..........  482K
   150K .......... .......... ..

  5350K .......... .......... .......... .......... .......... 1.31M
  5400K .......... .......... .......... .......... ..........  761K
  5450K .......... .......... .......... .......... .......... 76.3M
  5500K .......... .......... .......... .......... ..........  503K
  5550K .......... .......... .......... .......... .......... 13.8M
  5600K .......... .......... .......... .......... ..........  654K
  5650K .......... .......... .......... .......... .......... 1.85M
  5700K .......... .......... .......... .......... .......... 50.1M
  5750K .......... .......... .......... .......... ..........  491K
  5800K .......... .......... .......... .......... .......... 32.1M
  5850K .......... .......... .......... .......... ..........  661K
  5900K .......... .......... .......... .......... .......... 1.77M
  5950K .......... .......... .......... .......... ..........  281M
  6000K .......... .......... .......... .......... ..........  507K
  6050K .......... .......... ....

## Set up the remote codes

In [7]:
code_dir_path = code_remote_path+"/"+code_dir_name

In [8]:
# Eval mol. orbitals

# Python with MPI support and ASE
prepend_text  = "### code prepend_text start ###\n"
prepend_text += "module load cray-python/3.6.1.1\n"
prepend_text += "export PYTHONPATH=$PYTHONPATH:\"/users/keimre/soft/ase\"\n"
prepend_text += "### code prepend_text end ###\n"

code_label = "py_eval_morbs_"+version_id

code_full_name = code_label + "@" + computer_name

try:
    Code.get_from_string(code_full_name)
except (NotExistent, MultipleObjectsError):            
    code = Code(remote_computer_exec=(computer, code_dir_path + "/evaluate_morbs_in_box_mpi.py"))
    code.label = code_label
    code.description = "Python code to evaluate mol. orbitals on a grid"
    code.set_input_plugin_name("stm.eval_morbs")
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code.store()
    ! verdi code show "{code_full_name}"
else:
    print("Code '{}' already existent".format(code_full_name))       

 * PK:             3048
 * UUID:           faf0e9da-a88b-4986-87d5-2ca2b4bb2a49
 * Label:          py_eval_morbs_0eadfd
 * Description:    Python code to evaluate mol. orbitals on a grid
 * Default plugin: stm.eval_morbs
 * Used by:        0 calculations
 * Type:           remote
 * Remote machine: daint-s746-normal
 * Remote absolute path: 
   /users/keimre/aiida-soft/at_0eadfd/evaluate_morbs_in_box_mpi.py
 * prepend text:
   ### code prepend_text start ###
   module load cray-python/3.6.1.1
   export PYTHONPATH=$PYTHONPATH:"/users/keimre/soft/ase"
   ### code prepend_text end ###
   
 * append text:
   # No append text.


In [12]:
# stm & sts from npz

prepend_text  = "### code prepend_text start ###\n"
prepend_text += "export PATH='/users/keimre/soft/miniconda3/bin':$PATH\n"
prepend_text += "mkdir stm_output\n"
prepend_text += "### code prepend_text end ###\n"


code_label = "py_stm_sts_"+version_id

code_full_name = code_label + "@" + computer_name

try:
    Code.get_from_string(code_full_name)
except (NotExistent, MultipleObjectsError):            
    code = Code(remote_computer_exec=(computer, code_dir_path + "/stm_sts_extrap_from_npz.py"))
    code.label = code_label
    code.description = "Python code to calculate STM and STS from the output of py_eval_morbs"
    code.set_input_plugin_name("stm.stm_image")
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code.store()
    ! verdi code show "{code_full_name}"
else:
    print("Code '{}' already existent".format(code_full_name))     

 * PK:             3050
 * UUID:           e83e9557-0192-4cf8-8e83-55d3663067ef
 * Label:          py_stm_sts_0eadfd
 * Description:    Python code to calculate STM and STS from the output of py_eval_morbs
 * Default plugin: stm.stm_image
 * Used by:        0 calculations
 * Type:           remote
 * Remote machine: daint-s746-normal
 * Remote absolute path: 
   /users/keimre/aiida-soft/at_0eadfd/stm_sts_extrap_from_npz.py
 * prepend text:
   ### code prepend_text start ###
   export PATH='/users/keimre/soft/miniconda3/bin':$PATH
   mkdir stm_output
   ### code prepend_text end ###
   
 * append text:
   # No append text.


# Alternative: setup remote python

In [19]:
prepend_text  = "### code prepend_text start ###\n"
prepend_text += "module load cray-python/3.6.1.1\n"
prepend_text += "export PYTHONPATH=$PYTHONPATH:\"/users/keimre/soft/ase\"\n"
prepend_text += "### code prepend_text end ###\n"

exe_path = ! ssh {computer.hostname} "module load cray-python/3.6.1.1 && which python"

code_label = "python3"

code_full_name = code_label + "@" + computer_name

try:
    Code.get_from_string(code_full_name)
except (NotExistent, MultipleObjectsError):            
    code = Code(remote_computer_exec=(computer, exe_path[0]))
    code.label = code_label
    code.description = "Python 3 with custom libraries"
    code.set_prepend_text(prepend_text)
    code.set_append_text("")
    code.store()
    ! verdi code show "{code_full_name}"
else:
    print("Code '{}' already existent".format(code_full_name))     

 * PK:             2444
 * UUID:           8c7308b7-1cf8-4d53-b0d2-02ef70f345da
 * Label:          python3
 * Description:    Python 3 with custom libraries
 * Default plugin: None
 * Used by:        0 calculations
 * Type:           remote
 * Remote machine: daint-s746-normal
 * Remote absolute path: 
   /opt/python/3.6.1.1/bin/python
 * prepend text:
   ### code prepend_text start ###
   module load cray-python/3.6.1.1
   export PYTHONPATH=$PYTHONPATH:"/users/keimre/soft/ase"
   ### code prepend_text end ###
   
 * append text:
   # No append text.
